In [1]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import numpy as np

In [2]:
(train_data, train_target), (test_data, test_target) = boston_housing.load_data()

57344/57026 [==============================] - 0s 6us/step


In [4]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


### 數據預處理

標準化: $X_t = \frac{X-mean} {std} $

In [5]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data = (train_data - mean) / std
test_data = (test_data - mean) / std

### 模型

In [10]:
def build_model():
    model = Sequential()
    model.add(Dense(64, input_shape=(13,), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=RMSprop(), loss='mse', metrics=['mae'])
    return model

In [11]:
k=4
num_val_samples = len(train_data)//k
num_epochs = 100
all_scores = []

In [12]:
for i in range(k):
    val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_targets = train_target[i*num_val_samples:(i+1)*num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i*num_val_samples],
        train_data[(i+1)*num_val_samples:]],
        axis=0
    )
    partial_train_target = np.concatenate(
        [train_target[:i*num_val_samples],
        train_target[(i+1)*num_val_samples:]],
        axis=0
    )
    model = build_model()
    model.fit(partial_train_data, partial_train_target, epochs=num_epochs, batch_size = 1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
    print('第', i+1, '折', 'MSE', val_mse, 'MAE', val_mae)

Instructions for updating:
Use tf.cast instead.
第 1 折 MSE 7.0814119848874535 MAE 1.9199283123016357
第 2 折 MSE 12.647553245619973 MAE 2.777705192565918
第 3 折 MSE 14.828938635269013 MAE 2.634939670562744
第 4 折 MSE 13.16494856022372 MAE 2.5297834873199463


In [13]:
np.mean(all_scores)

2.465589165687561

In [15]:
model = build_model()
model.fit(train_data, train_target, epochs=80, batch_size=16, verbose=0)

In [17]:
test_mse, test_mae = model.evaluate(test_data, test_target)
print(np.sqrt(test_mse))

102/102 [==============================] - 0s 78us/step
4.09944643345502


### 小結:
* 根據Kaggle上Boston Housing的評估指標RMSE，本模型測試可排到Leaderboard約24名的位置。